<a href="https://colab.research.google.com/github/dashsumit/Automated_Text_Summerization-/blob/main/Bengali_Text_Summerizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
!pip install bnlp-toolkit

In [63]:
!pip install fasttext

In [64]:
from bnlp import SentencepieceTokenizer
from bnlp import BasicTokenizer # ['আমি', 'বাংলায়', 'গান', 'গাই', '।'] from bnlp import NLTKTokenizer

from bnlp import BengaliCorpus as corpus
from bnlp import BengaliPOS
from collections import Counter
from bnlp import NLTKTokenizer
import pandas as pd
from heapq import nlargest,nsmallest
from bnlp.embedding.fasttext import BengaliFasttext
from transformers import pipeline

In [65]:
print(corpus.stopwords)
print(corpus.punctuations)
print(corpus.letters)
print(corpus.digits)
print(corpus.vowels)

['অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই', 'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও'

In [83]:
text = '''
অত্যাধুনিক উদ্ভাবনের হৃদয়ে চলছে এক নীরব বিপ্লব—কৃত্রিম বুদ্ধিমত্তা । এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স্বপ্নের বাস্তব রূপ: যেখানে যন্ত্রগুলি মানুষের বুদ্ধিমত্তা অনুকরণ করতে পারে, এমনকি সেটিকে অতিক্রমও করতে পারে। কৃত্রিম বুদ্ধিমত্তা এর মূল উদ্দেশ্য হলো যন্ত্রকে শেখানো—ভাবতে, শিখতে এবং মানিয়ে নিতে—যেমনটা আমরা করি।

কিন্তু এটা কীভাবে সম্ভব? এর উত্তর লুকিয়ে আছে কৃত্রিম বুদ্ধিমত্তা এর প্রাণে মেশিন লার্নিং । মেশিন লার্নিং এমন একটি পদ্ধতি যার মাধ্যমে যন্ত্র ডেটা থেকে শেখে। যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব্দ বলতে কিংবা ধাঁধা সমাধান করতে শেখে, তেমনি একটি যন্ত্রও উদাহরণ, প্যাটার্ন এবং ফলাফল থেকে শেখে।

কৃত্রিম বুদ্ধিমত্তা শুধুমাত্র শেখার মধ্যে সীমাবদ্ধ নয়; এটি উপলব্ধি করা, যুক্তি তৈরি করা এবং কাজ করার মধ্যেও বিস্তৃত। ন্যাচারাল ল্যাংগুয়েজ প্রসেসিং (এনলপি ) এর মাধ্যমে যন্ত্র মানব ভাষা বুঝতে পারে। কম্পিউটার ভিশন এর মাধ্যমে তারা ছবি এবং দৃশ্য বিশ্লেষণ করতে পারে। নিউরাল নেটওয়ার্ক ব্যবহার করে তারা মানব মস্তিষ্কের কাঠামো অনুকরণ করে—জটিল ডেটা প্রক্রিয়াকরণ করে "নিউরন"-এর স্তরের মধ্য দিয়ে।

 মেশিন লার্নিং এর সবচেয়ে শক্তিশালী আর্কিটেকচারের একটি হলো ডিপ লার্নিং মডেল, যেখানে যন্ত্র বিভিন্ন স্তরের মাধ্যমে বিমূর্ত বৈশিষ্ট্য শিখে। এটি সেই প্রযুক্তি যা ভয়েস অ্যাসিস্ট্যান্ট, স্মার্ট অনুবাদক এবং এমনকি শিল্পকর্ম তৈরিতেও ব্যবহৃত হয়।

তবে অ্যালগরিদম বা মডেলের বাইরেও, কৃত্রিম বুদ্ধিমত্তা আমাদেরই প্রতিচ্ছবি—আমাদের সৃজনশীলতা, যুক্তি এবং বুদ্ধিমত্তার প্রতি অন্বেষণ। শক্তির সঙ্গে আসে দায়িত্ব, এবং নৈতিক কৃত্রিম বুদ্ধিমত্তা আমাদের মনে করিয়ে দেয়, আমাদের এমন একটি ভবিষ্যৎ গড়তে হবে যা সবার জন্য ন্যায্য, স্বচ্ছ এবং কল্যাণকর।

মেশিন লার্নিং এবং কৃত্রিম বুদ্ধিমত্তা শুধুই প্রযুক্তি নয়—তারা হলো এক সেতুবন্ধন মানুষের মন এবং যন্ত্রের জগৎ-এর মধ্যে, যা জীবনের প্রতিটি দিককে রূপান্তর করার সম্ভাবনা রাখে—চিকিৎসা থেকে সঙ্গীত, শিক্ষা থেকে অনুসন্ধান পর্যন্ত।
'''

In [84]:
text

'\nঅত্যাধুনিক উদ্ভাবনের হৃদয়ে চলছে এক নীরব বিপ্লব—কৃত্রিম বুদ্ধিমত্তা । এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স্বপ্নের বাস্তব রূপ: যেখানে যন্ত্রগুলি মানুষের বুদ্ধিমত্তা অনুকরণ করতে পারে, এমনকি সেটিকে অতিক্রমও করতে পারে। কৃত্রিম বুদ্ধিমত্তা এর মূল উদ্দেশ্য হলো যন্ত্রকে শেখানো—ভাবতে, শিখতে এবং মানিয়ে নিতে—যেমনটা আমরা করি।\n\nকিন্তু এটা কীভাবে সম্ভব? এর উত্তর লুকিয়ে আছে কৃত্রিম বুদ্ধিমত্তা এর প্রাণে মেশিন লার্নিং । মেশিন লার্নিং এমন একটি পদ্ধতি যার মাধ্যমে যন্ত্র ডেটা থেকে শেখে। যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব্দ বলতে কিংবা ধাঁধা সমাধান করতে শেখে, তেমনি একটি যন্ত্রও উদাহরণ, প্যাটার্ন এবং ফলাফল থেকে শেখে।\n\nকৃত্রিম বুদ্ধিমত্তা শুধুমাত্র শেখার মধ্যে সীমাবদ্ধ নয়; এটি উপলব্ধি করা, যুক্তি তৈরি করা এবং কাজ করার মধ্যেও বিস্তৃত। ন্যাচারাল ল্যাংগুয়েজ প্রসেসিং (এনলপি ) এর মাধ্যমে যন্ত্র মানব ভাষা বুঝতে পারে। কম্পিউটার ভিশন এর মাধ্যমে তারা ছবি এবং দৃশ্য বিশ্লেষণ করতে পারে। নিউরাল নেটওয়ার্ক ব্যবহার করে তারা মানব মস্তিষ্কের কাঠামো অনুকরণ করে—জটিল ডেটা প্রক্রিয়াকরণ করে "নিউ

In [85]:
len(text)

1770

Tokenizing the text into tokens

In [86]:
bsp = SentencepieceTokenizer()

tokens = bsp.tokenize(text)
tokens

['▁অত্যাধুনিক',
 '▁উদ্ভাবনের',
 '▁হৃদয়ে',
 '▁চলছে',
 '▁এক',
 '▁নীরব',
 '▁বিপ্লব',
 '—',
 'কৃত্রিম',
 '▁বুদ্ধিমত্তা',
 '▁।',
 '▁এটি',
 '▁শুধু',
 'ই',
 '▁একটি',
 '▁বৈজ্ঞানিক',
 '▁ক্ষেত্র',
 '▁নয়',
 ',',
 '▁বরং',
 '▁এক',
 '▁স্বপ্নের',
 '▁বাস্তব',
 '▁রূপ',
 ':',
 '▁যেখানে',
 '▁যন্ত্র',
 'গুলি',
 '▁মানুষের',
 '▁বুদ্ধিমত্তা',
 '▁অনুকরণ',
 '▁করতে',
 '▁পারে',
 ',',
 '▁এমনকি',
 '▁সেটিকে',
 '▁অতিক্রম',
 'ও',
 '▁করতে',
 '▁পারে',
 '।',
 '▁কৃত্রিম',
 '▁বুদ্ধিমত্তা',
 '▁এর',
 '▁মূল',
 '▁উদ্দেশ্য',
 '▁হলো',
 '▁যন্ত্র',
 'কে',
 '▁শেখানো',
 '—',
 'ভাব',
 'তে',
 ',',
 '▁শিখতে',
 '▁এবং',
 '▁মানিয়ে',
 '▁নিতে',
 '—',
 'যেমন',
 'টা',
 '▁আমরা',
 '▁করি',
 '।',
 '▁কিন্তু',
 '▁এটা',
 '▁কীভাবে',
 '▁সম্ভব',
 '?',
 '▁এর',
 '▁উত্তর',
 '▁লুকিয়ে',
 '▁আছে',
 '▁কৃত্রিম',
 '▁বুদ্ধিমত্তা',
 '▁এর',
 '▁প্রাণে',
 '▁মেশিন',
 '▁লার্নিং',
 '▁।',
 '▁মেশিন',
 '▁লার্নিং',
 '▁এমন',
 '▁একটি',
 '▁পদ্ধতি',
 '▁যার',
 '▁মাধ্যমে',
 '▁যন্ত্র',
 '▁ডেটা',
 '▁থেকে',
 '▁শেখে',
 '।',
 '▁যেমন',
 '▁একটি',
 '▁শিশু',
 '▁অভিজ্ঞতার',
 '▁মাধ্যমে

In [87]:
len(tokens)

327

Removing the stopwords from tokens

In [88]:
stopwords = corpus.stopwords
filtered_tokens = [token for token in tokens if token not in stopwords]
filtered_tokens

['▁অত্যাধুনিক',
 '▁উদ্ভাবনের',
 '▁হৃদয়ে',
 '▁চলছে',
 '▁এক',
 '▁নীরব',
 '▁বিপ্লব',
 '—',
 'কৃত্রিম',
 '▁বুদ্ধিমত্তা',
 '▁।',
 '▁এটি',
 '▁শুধু',
 '▁একটি',
 '▁বৈজ্ঞানিক',
 '▁ক্ষেত্র',
 '▁নয়',
 ',',
 '▁বরং',
 '▁এক',
 '▁স্বপ্নের',
 '▁বাস্তব',
 '▁রূপ',
 ':',
 '▁যেখানে',
 '▁যন্ত্র',
 '▁মানুষের',
 '▁বুদ্ধিমত্তা',
 '▁অনুকরণ',
 '▁করতে',
 '▁পারে',
 ',',
 '▁এমনকি',
 '▁সেটিকে',
 '▁অতিক্রম',
 '▁করতে',
 '▁পারে',
 '।',
 '▁কৃত্রিম',
 '▁বুদ্ধিমত্তা',
 '▁এর',
 '▁মূল',
 '▁উদ্দেশ্য',
 '▁হলো',
 '▁যন্ত্র',
 '▁শেখানো',
 '—',
 'ভাব',
 'তে',
 ',',
 '▁শিখতে',
 '▁এবং',
 '▁মানিয়ে',
 '▁নিতে',
 '—',
 'টা',
 '▁আমরা',
 '▁করি',
 '।',
 '▁কিন্তু',
 '▁এটা',
 '▁কীভাবে',
 '▁সম্ভব',
 '?',
 '▁এর',
 '▁উত্তর',
 '▁লুকিয়ে',
 '▁আছে',
 '▁কৃত্রিম',
 '▁বুদ্ধিমত্তা',
 '▁এর',
 '▁প্রাণে',
 '▁মেশিন',
 '▁লার্নিং',
 '▁।',
 '▁মেশিন',
 '▁লার্নিং',
 '▁এমন',
 '▁একটি',
 '▁পদ্ধতি',
 '▁যার',
 '▁মাধ্যমে',
 '▁যন্ত্র',
 '▁ডেটা',
 '▁থেকে',
 '▁শেখে',
 '।',
 '▁যেমন',
 '▁একটি',
 '▁শিশু',
 '▁অভিজ্ঞতার',
 '▁মাধ্যমে',
 '▁মুখ',
 '▁চিনতে',
 ',',
 '▁শব্দ'

In [89]:
len(filtered_tokens)

311

Frequency Of Each Sentences

In [90]:
word_freq=Counter(filtered_tokens)
word_freq

Counter({'▁অত্যাধুনিক': 1,
         '▁উদ্ভাবনের': 1,
         '▁হৃদয়ে': 1,
         '▁চলছে': 1,
         '▁এক': 3,
         '▁নীরব': 1,
         '▁বিপ্লব': 1,
         '—': 7,
         'কৃত্রিম': 1,
         '▁বুদ্ধিমত্তা': 8,
         '▁।': 2,
         '▁এটি': 3,
         '▁শুধু': 2,
         '▁একটি': 6,
         '▁বৈজ্ঞানিক': 1,
         '▁ক্ষেত্র': 1,
         '▁নয়': 3,
         ',': 16,
         '▁বরং': 1,
         '▁স্বপ্নের': 1,
         '▁বাস্তব': 1,
         '▁রূপ': 1,
         ':': 1,
         '▁যেখানে': 2,
         '▁যন্ত্র': 6,
         '▁মানুষের': 2,
         '▁অনুকরণ': 2,
         '▁করতে': 4,
         '▁পারে': 4,
         '▁এমনকি': 2,
         '▁সেটিকে': 1,
         '▁অতিক্রম': 1,
         '।': 13,
         '▁কৃত্রিম': 6,
         '▁এর': 6,
         '▁মূল': 1,
         '▁উদ্দেশ্য': 1,
         '▁হলো': 3,
         '▁শেখানো': 1,
         'ভাব': 1,
         'তে': 1,
         '▁শিখতে': 1,
         '▁এবং': 10,
         '▁মানিয়ে': 1,
         '▁নিতে': 1,
         'টা': 1,
   

Maximum Freq Of Each Sentences

In [91]:
max_freq= max(word_freq.values())
max_freq

16

Normalizing The Values

In [92]:
for word in word_freq.keys():
  word_freq[word] = word_freq[word] / max_freq

word_freq

Counter({'▁অত্যাধুনিক': 0.0625,
         '▁উদ্ভাবনের': 0.0625,
         '▁হৃদয়ে': 0.0625,
         '▁চলছে': 0.0625,
         '▁এক': 0.1875,
         '▁নীরব': 0.0625,
         '▁বিপ্লব': 0.0625,
         '—': 0.4375,
         'কৃত্রিম': 0.0625,
         '▁বুদ্ধিমত্তা': 0.5,
         '▁।': 0.125,
         '▁এটি': 0.1875,
         '▁শুধু': 0.125,
         '▁একটি': 0.375,
         '▁বৈজ্ঞানিক': 0.0625,
         '▁ক্ষেত্র': 0.0625,
         '▁নয়': 0.1875,
         ',': 1.0,
         '▁বরং': 0.0625,
         '▁স্বপ্নের': 0.0625,
         '▁বাস্তব': 0.0625,
         '▁রূপ': 0.0625,
         ':': 0.0625,
         '▁যেখানে': 0.125,
         '▁যন্ত্র': 0.375,
         '▁মানুষের': 0.125,
         '▁অনুকরণ': 0.125,
         '▁করতে': 0.25,
         '▁পারে': 0.25,
         '▁এমনকি': 0.125,
         '▁সেটিকে': 0.0625,
         '▁অতিক্রম': 0.0625,
         '।': 0.8125,
         '▁কৃত্রিম': 0.375,
         '▁এর': 0.375,
         '▁মূল': 0.0625,
         '▁উদ্দেশ্য': 0.0625,
         '▁হলো': 0.1875,
 

Sentence Tokenizer

In [93]:
import nltk
nltk.download('punkt_tab')

bnltk = NLTKTokenizer()

sentence_tokens = bnltk.sentence_tokenize(text)
sentence_tokens

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


['\nঅত্যাধুনিক উদ্ভাবনের হৃদয়ে চলছে এক নীরব বিপ্লব—কৃত্রিম বুদ্ধিমত্তা ।',
 'এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স্বপ্নের বাস্তব রূপ: যেখানে যন্ত্রগুলি মানুষের বুদ্ধিমত্তা অনুকরণ করতে পারে, এমনকি সেটিকে অতিক্রমও করতে পারে।',
 'কৃত্রিম বুদ্ধিমত্তা এর মূল উদ্দেশ্য হলো যন্ত্রকে শেখানো—ভাবতে, শিখতে এবং মানিয়ে নিতে—যেমনটা আমরা করি।',
 'কিন্তু এটা কীভাবে সম্ভব?',
 'এর উত্তর লুকিয়ে আছে কৃত্রিম বুদ্ধিমত্তা এর প্রাণে মেশিন লার্নিং ।',
 'মেশিন লার্নিং এমন একটি পদ্ধতি যার মাধ্যমে যন্ত্র ডেটা থেকে শেখে।',
 'যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব্দ বলতে কিংবা ধাঁধা সমাধান করতে শেখে, তেমনি একটি যন্ত্রও উদাহরণ, প্যাটার্ন এবং ফলাফল থেকে শেখে।',
 'কৃত্রিম বুদ্ধিমত্তা শুধুমাত্র শেখার মধ্যে সীমাবদ্ধ নয়; এটি উপলব্ধি করা, যুক্তি তৈরি করা এবং কাজ করার মধ্যেও বিস্তৃত।',
 'ন্যাচারাল ল্যাংগুয়েজ প্রসেসিং (এনলপি ) এর মাধ্যমে যন্ত্র মানব ভাষা বুঝতে পারে।',
 'কম্পিউটার ভিশন এর মাধ্যমে তারা ছবি এবং দৃশ্য বিশ্লেষণ করতে পারে।',
 'নিউরাল নেটওয়ার্ক ব্যবহার করে তারা মানব মস্তিষ্কের কাঠামো অনুকরণ ক

In [94]:
len(sentence_tokens)

16

writing logic to calculate sentences score based on word frequencies

In [95]:
sent_score = {}  # Initialize an empty dictionary to store sentence scores

for sentence in sentence_tokens:  # Iterate through each sentence
    for word in bsp.tokenize(sentence):  # Tokenize the sentence into words
        if word in word_freq.keys():  # Check if the word is in the word frequency dictionary
            if sentence in sent_score.keys():  # Check if the sentence is already in the sentence score dictionary
                sent_score[sentence] += word_freq[word]  # Add the word's frequency to the sentence score
            else:
                sent_score[sentence] = word_freq[word]  # Initialize the sentence score with the word's frequency




In [96]:
sent_score

{'\nঅত্যাধুনিক উদ্ভাবনের হৃদয়ে চলছে এক নীরব বিপ্লব—কৃত্রিম বুদ্ধিমত্তা ।': 1.6875,
 'এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স্বপ্নের বাস্তব রূপ: যেখানে যন্ত্রগুলি মানুষের বুদ্ধিমত্তা অনুকরণ করতে পারে, এমনকি সেটিকে অতিক্রমও করতে পারে।': 6.8125,
 'কৃত্রিম বুদ্ধিমত্তা এর মূল উদ্দেশ্য হলো যন্ত্রকে শেখানো—ভাবতে, শিখতে এবং মানিয়ে নিতে—যেমনটা আমরা করি।': 5.8125,
 'কিন্তু এটা কীভাবে সম্ভব?': 0.3125,
 'এর উত্তর লুকিয়ে আছে কৃত্রিম বুদ্ধিমত্তা এর প্রাণে মেশিন লার্নিং ।': 2.5625,
 'মেশিন লার্নিং এমন একটি পদ্ধতি যার মাধ্যমে যন্ত্র ডেটা থেকে শেখে।': 3.25,
 'যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব্দ বলতে কিংবা ধাঁধা সমাধান করতে শেখে, তেমনি একটি যন্ত্রও উদাহরণ, প্যাটার্ন এবং ফলাফল থেকে শেখে।': 7.625,
 'কৃত্রিম বুদ্ধিমত্তা শুধুমাত্র শেখার মধ্যে সীমাবদ্ধ নয়; এটি উপলব্ধি করা, যুক্তি তৈরি করা এবং কাজ করার মধ্যেও বিস্তৃত।': 4.875,
 'ন্যাচারাল ল্যাংগুয়েজ প্রসেসিং (এনলপি ) এর মাধ্যমে যন্ত্র মানব ভাষা বুঝতে পারে।': 2.8125,
 'কম্পিউটার ভিশন এর মাধ্যমে তারা ছবি এবং দৃশ্য বিশ্লেষণ করতে পারে।': 

Creating a dataframe in pandas

In [97]:
df =pd.DataFrame(sent_score.items(),columns=['sentence','score'])
df

,sentence,score
0,\nঅত্যাধুনিক উদ্ভাবনের হৃদয়ে চলছে এক নীরব বিপ...,1.6875
1,"এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স...",6.8125
2,কৃত্রিম বুদ্ধিমত্তা এর মূল উদ্দেশ্য হলো যন্ত্র...,5.8125
3,কিন্তু এটা কীভাবে সম্ভব?,0.3125
4,এর উত্তর লুকিয়ে আছে কৃত্রিম বুদ্ধিমত্তা এর প্...,2.5625
5,মেশিন লার্নিং এমন একটি পদ্ধতি যার মাধ্যমে যন্ত...,3.2500
6,"যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব...",7.6250
7,কৃত্রিম বুদ্ধিমত্তা শুধুমাত্র শেখার মধ্যে সীমা...,4.8750
8,ন্যাচারাল ল্যাংগুয়েজ প্রসেসিং (এনলপি ) এর মাধ...,2.8125
9,কম্পিউটার ভিশন এর মাধ্যমে তারা ছবি এবং দৃশ্য ব...,3.0625


In [102]:
number_of_sentences = 4
n= nlargest(number_of_sentences,sent_score,key =sent_score.get)
n

# number_of_sentences = 4  # Specify the number of sentences to select
# n = nsmallest(number_of_sentences, sent_score, key=sent_score.get)  # Use nsmallest to get sentences with lowest scores

['মেশিন লার্নিং এবং কৃত্রিম বুদ্ধিমত্তা শুধুই প্রযুক্তি নয়—তারা হলো এক সেতুবন্ধন মানুষের মন এবং যন্ত্রের জগৎ-এর মধ্যে, যা জীবনের প্রতিটি দিককে রূপান্তর করার সম্ভাবনা রাখে—চিকিৎসা থেকে সঙ্গীত, শিক্ষা থেকে অনুসন্ধান পর্যন্ত।',
 'শক্তির সঙ্গে আসে দায়িত্ব, এবং নৈতিক কৃত্রিম বুদ্ধিমত্তা আমাদের মনে করিয়ে দেয়, আমাদের এমন একটি ভবিষ্যৎ গড়তে হবে যা সবার জন্য ন্যায্য, স্বচ্ছ এবং কল্যাণকর।',
 'যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব্দ বলতে কিংবা ধাঁধা সমাধান করতে শেখে, তেমনি একটি যন্ত্রও উদাহরণ, প্যাটার্ন এবং ফলাফল থেকে শেখে।',
 'এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স্বপ্নের বাস্তব রূপ: যেখানে যন্ত্রগুলি মানুষের বুদ্ধিমত্তা অনুকরণ করতে পারে, এমনকি সেটিকে অতিক্রমও করতে পারে।']

In [103]:
# Print the selected sentences or perform further processing
for sentence in n:
    print(sentence)


মেশিন লার্নিং এবং কৃত্রিম বুদ্ধিমত্তা শুধুই প্রযুক্তি নয়—তারা হলো এক সেতুবন্ধন মানুষের মন এবং যন্ত্রের জগৎ-এর মধ্যে, যা জীবনের প্রতিটি দিককে রূপান্তর করার সম্ভাবনা রাখে—চিকিৎসা থেকে সঙ্গীত, শিক্ষা থেকে অনুসন্ধান পর্যন্ত।
শক্তির সঙ্গে আসে দায়িত্ব, এবং নৈতিক কৃত্রিম বুদ্ধিমত্তা আমাদের মনে করিয়ে দেয়, আমাদের এমন একটি ভবিষ্যৎ গড়তে হবে যা সবার জন্য ন্যায্য, স্বচ্ছ এবং কল্যাণকর।
যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব্দ বলতে কিংবা ধাঁধা সমাধান করতে শেখে, তেমনি একটি যন্ত্রও উদাহরণ, প্যাটার্ন এবং ফলাফল থেকে শেখে।
এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স্বপ্নের বাস্তব রূপ: যেখানে যন্ত্রগুলি মানুষের বুদ্ধিমত্তা অনুকরণ করতে পারে, এমনকি সেটিকে অতিক্রমও করতে পারে।


In [104]:
" ".join(n)

'মেশিন লার্নিং এবং কৃত্রিম বুদ্ধিমত্তা শুধুই প্রযুক্তি নয়—তারা হলো এক সেতুবন্ধন মানুষের মন এবং যন্ত্রের জগৎ-এর মধ্যে, যা জীবনের প্রতিটি দিককে রূপান্তর করার সম্ভাবনা রাখে—চিকিৎসা থেকে সঙ্গীত, শিক্ষা থেকে অনুসন্ধান পর্যন্ত। শক্তির সঙ্গে আসে দায়িত্ব, এবং নৈতিক কৃত্রিম বুদ্ধিমত্তা আমাদের মনে করিয়ে দেয়, আমাদের এমন একটি ভবিষ্যৎ গড়তে হবে যা সবার জন্য ন্যায্য, স্বচ্ছ এবং কল্যাণকর। যেমন একটি শিশু অভিজ্ঞতার মাধ্যমে মুখ চিনতে, শব্দ বলতে কিংবা ধাঁধা সমাধান করতে শেখে, তেমনি একটি যন্ত্রও উদাহরণ, প্যাটার্ন এবং ফলাফল থেকে শেখে। এটি শুধুই একটি বৈজ্ঞানিক ক্ষেত্র নয়, বরং এক স্বপ্নের বাস্তব রূপ: যেখানে যন্ত্রগুলি মানুষের বুদ্ধিমত্তা অনুকরণ করতে পারে, এমনকি সেটিকে অতিক্রমও করতে পারে।'